In [2]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
import keras_metrics as km # for precsion/ recall metrics
from keras import backend as K
from keras.callbacks import ModelCheckpoint # to save best model
from keras.models import load_model

from sklearn.model_selection import train_test_split

import glob
import json
import matplotlib.pyplot as plt

# for openvino
import tensorflow as tf

Using TensorFlow backend.
/home/evgenii/anaconda3/envs/my_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/evgenii/anaconda3/envs/my_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/evgenii/anaconda3/envs/my_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/evgenii/ana

In [3]:
#%%
from data_generator_sx3 import SX3Dataset
from data_generator import CorrDatasetV2
from model import Model
from utils import save_model#, load_model

### Load data

In [5]:
#%% prepare sx3 data (only module)
dataset_mp = SX3Dataset(label=1, global_path='sx3_data/outputs/mp')
dataset_nomp = SX3Dataset(label=0, global_path='sx3_data/outputs/no_mp')

data_mp = dataset_mp.build()
data_nomp = dataset_nomp.build()[:500]

dataset = np.concatenate((data_mp, data_nomp), axis=0)
np.random.shuffle(dataset)

data_train, data_val = train_test_split(dataset, test_size=0.2)

# 1 channel image (only module), add newaxis
X_train_sx = np.array([x['table'] for x in data_train])
X_val_sx = np.array([x['table'] for x in data_val])

y_train_sx = np.array([x['label'] for x in data_train])
y_val_sx = np.array([x['label'] for x in data_val])

In [6]:
# check shapes
X_train_sx.shape, X_val_sx.shape

((176, 40, 40, 2), (44, 40, 40, 2))

In [7]:
#%% prepare data generator data (only module)
global_path_mp_i = 'synth_data/mp/*_i_*'
global_path_mp_q = 'synth_data/mp/*_q_*'
global_path_nomp_i = 'synth_data/no_mp/*_i_*'
global_path_nomp_q = 'synth_data/no_mp/*_q_*'
paths_mp_i = sorted(glob.glob(global_path_mp_i))
paths_mp_q = sorted(glob.glob(global_path_mp_q))
paths_nomp_i = sorted(glob.glob(global_path_nomp_i))
paths_nomp_q = sorted(glob.glob(global_path_nomp_q))

synth_data_samples = []
synth_data_labels = []
for path_mp_i, path_mp_q in zip(paths_mp_i, paths_mp_q):
    matr_i = pd.read_csv(path_mp_i, sep=',', header=None).values
    matr_q = pd.read_csv(path_mp_q, sep=',', header=None).values
    matr_i = matr_i[...,None]
    matr_q = matr_q[...,None]
    matr = np.concatenate((matr_i, matr_q), axis=2)
    #matr = matr_i**2 + matr_q**2
    synth_data_samples.append(matr)
    synth_data_labels.append(1)
    
for path_nomp_i, path_nomp_q in zip(paths_nomp_i, paths_nomp_q):
    matr_i = pd.read_csv(path_nomp_i, sep=',', header=None).values
    matr_q = pd.read_csv(path_nomp_q, sep=',', header=None).values
    matr_i = matr_i[...,None]
    matr_q = matr_q[...,None]
    matr = np.concatenate((matr_i, matr_q), axis=2)
    #matr = matr_i**2 + matr_q**2
    synth_data_samples.append(matr)
    synth_data_labels.append(0)

synth_data_samples = np.array(synth_data_samples)
synth_data_labels = np.array(synth_data_labels)

X_train_synth, X_val_synth, y_train_synth, y_val_synth = train_test_split(synth_data_samples, synth_data_labels, test_size=0.2, shuffle=True)


### Build, train, save model

In [21]:
#%% Define model.
model = Model(shape=(X_train_synth.shape[1], X_train_synth.shape[2], X_train_synth.shape[3]))

batch_size = 8
train_iters = 10
learning_rate = 1e-4

model.model.compile(loss='binary_crossentropy',
					optimizer=optimizers.Adam(lr=learning_rate),
					metrics=['acc',
						   km.binary_precision(),
						   km.binary_recall()])

tracking <tf.Variable 'Variable_8:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_9:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_10:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_11:0' shape=() dtype=int32> fn


In [22]:
# load model from .h5
#model.model = load_model('saved_models/sc1_data_gen_train.pkl')
#model.model = load_model('saved_models/sc2_fine_tune.pkl')

In [23]:
# train model, sae best weigths with checkpointer
checkpointer = ModelCheckpoint(filepath='saved_models/best_model/best_mp_model.h5', verbose=1, save_best_only=True)

#%% Train model: pretrain on data gen
history = model.model.fit(
    x=X_train_synth,
    y=y_train_synth,
    validation_data=(X_val_synth, y_val_synth),
    epochs=train_iters,
    batch_size=batch_size,
    callbacks=[checkpointer]
    )
#save_model(model.model, 'saved_models/sc1_data_gen_train_zoom.pkl')

Train on 480 samples, validate on 120 samples
Epoch 1/10
480/480 [==============================] - 5s 10ms/step - loss: 0.6913 - acc: 0.5208 - precision: 0.5126 - recall: 0.8755 - val_loss: 0.6849 - val_acc: 0.6833 - val_precision: 0.9162 - val_recall: 0.4019

Epoch 00001: val_loss improved from inf to 0.68491, saving model to saved_models/best_model/best_mp_model.h5
Epoch 2/10
480/480 [==============================] - 4s 8ms/step - loss: 0.6809 - acc: 0.5604 - precision: 0.5629 - recall: 0.8003 - val_loss: 0.6759 - val_acc: 0.4500 - val_precision: 0.4898 - val_recall: 1.0000

Epoch 00002: val_loss improved from 0.68491 to 0.67591, saving model to saved_models/best_model/best_mp_model.h5
Epoch 3/10
480/480 [==============================] - 4s 8ms/step - loss: 0.6554 - acc: 0.6354 - precision: 0.5509 - recall: 0.9809 - val_loss: 0.6335 - val_acc: 0.6917 - val_precision: 0.7414 - val_recall: 0.7045

Epoch 00003: val_loss improved from 0.67591 to 0.63349, saving model to saved_models/b

### Convert Keras model .h5 to TF frozen graph .pb

In [25]:
# keras to tf conversion

keras_model_path = 'saved_models/best_model/best_mp_model.h5'

# freeze state of a session into a pruned compilation graph
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = convert_variables_to_constants(session, input_graph_def, 
                                                      output_names, freeze_var_names)
        return frozen_graph
    
# loading keras model
K.set_learning_phase(0)
model = load_model(keras_model_path,
                  custom_objects={
                      'binary_precision': km.binary_precision(),
                      'binary_recall': km.binary_recall()
                  })

# create frozen graph of the keras model
frozen_graph = freeze_session(K.get_session(), 
                              output_names=[out.op.name for out in model.outputs])

# save model as .pb file
tf.train.write_graph(frozen_graph, 'saved_models/tf_model', 'tf_mp_model.pb', as_text=False)

tracking <tf.Variable 'Variable_14:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_15:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_16:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_17:0' shape=() dtype=int32> fn
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 292 variables.
INFO:tensorflow:Converted 292 variables to const ops.


'saved_models/tf_model/tf_mp_model.pb'

### Convert TF .pb to IR model

In [72]:
# convert tensorflow graph to ir_model (from terminal or bash)
! chmod 744 convert_tf_ncs2.sh
! ./convert_tf_ncs2.sh

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/evgenii/Documents/01_These/gnss_signal_generator/saved_models/tf_model/tf_mp_model.pb
	- Path for generated IR: 	/home/evgenii/Documents/01_These/gnss_signal_generator/saved_models/ir_model
	- IR output name: 	ir_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,40,40,2]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- 

### Make inference on NCS2

In [70]:
print(X_train_sx[0].shape)
prepimg = np.ndarray(shape=(n,c,h,w))
prepimg[0, 0, :,:] = np.moveaxis(X_train_sx[0], -1, 0)[0,:]
print(prepimg.shape)

(40, 40, 2)
(1, 2, 40, 40)


In [66]:
np.moveaxis(X_train_sx[0], -1, 0)[0,:].shape

(40, 40)

In [23]:
# run inference on NCS2
import cv2
import sys
import os
import logging as log
from time import time
from openvino.inference_engine import IENetwork, IEPlugin

log.basicConfig(format='[ %(levelname)s ] %(message)s', level=log.INFO, stream=sys.stdout)
model_xml_path = 'saved_models/ir_model/ir_model.xml'
model_bin_path = 'saved_models/ir_model/ir_model.bin'

# plugin initialization for specified device
plugin = IEPlugin(device='MYRIAD')

# read ir model
log.info('loading network files: \n\t{}\n\t{}'.format(model_xml_path, model_bin_path))
net = IENetwork(model=model_xml_path, weights=model_bin_path)

log.info('Preparing input blobs')
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))

# prepare image
# get shapes
n, c, h, w = net.inputs[input_blob].shape
log.info('required img shape:  {},{},{},{}'.format(n, c, h, w))
prepimg = np.ndarray(shape=(n, c, h, w))

# change data layout from HW to NCHW
prepimg[0,0,:,:] = np.moveaxis(X_train_sx[0], -1, 0)[0,:]
log.info('prepimg shape:  {},{},{},{}'.format(prepimg.shape[0], prepimg.shape[1], prepimg.shape[2], prepimg.shape[3]))



[ INFO ] loading network files: 
	saved_models/ir_model/ir_model.xml
	saved_models/ir_model/ir_model.bin
[ INFO ] Preparing input blobs
[ INFO ] required img shape:  1,2,40,40
[ INFO ] prepimg shape:  1,2,40,40


In [24]:
# loading model to plugin
log.info('Loading model to the plugin')
exec_net = plugin.load(network=net)
del net

# start sync inference
log.info('Starting inference ({} iterations)'.format(1))
infer_time = []
t0 = time()
res = exec_net.infer(inputs={input_blob: prepimg})


[ INFO ] Loading model to the plugin
[ INFO ] Starting inference (1 iterations)


In [13]:
exec_net

-------------------------------

In [25]:
infer_time.append((time() - t0) * 1000)
log.info('Average running time of one iteration: {} ms'.format(np.average(np.asarray(infer_time))))


[ INFO ] Average running time of one iteration: 21335.390329360962 ms


In [26]:
# processing output blob
log.info('Processing output blob')
res = res[out_blob]
print(res)

del exec_net
del plugin

[ INFO ] Processing output blob
[[0.]]
